In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import cv2
import os
import time 

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
%matplotlib inline

### Data Augmentation For Behavioral Cloning 

In [ ]:

def pre_process_image():
    
    ang_range = 20
    shear_range = 10
    trans_range = 5
    
    X_aug_images = []
    Y_steer_angles = []
    
    driving_log_df = pd.read_csv('driving_log.csv')

    for j in range(1):

        ang_range += j*5
        shear_range += j*2
        trans_range += j*3

        print ('Epoch ',j,'Started',time.time())

        for i in range (driving_log_df.shape[0]) :

            steering = driving_log_df['steering'][i]

            centre_img = driving_log_df['center'][i]
            centre_img = centre_img.strip()

            left_img = driving_log_df['left'][i]
            left_img = left_img.strip()

            right_img = driving_log_df['right'][i]
            right_img = right_img.strip()

            centre_img = get_augmented_image(centre_img,ang_range,shear_range,trans_range)
            X_aug_images.append(centre_img)
            Y_steer_angles.append(steering)


            left_img = get_augmented_image(left_img,ang_range,shear_range,trans_range)
            X_aug_images.append(left_img)
            Y_steer_angles.append(steering)

            right_img = get_augmented_image(right_img,ang_range,shear_range,trans_range)
            X_aug_images.append(right_img)
            Y_steer_angles.append(steering)

            if(i % 500 ==0):

                print ('No Of image Augmented ',i,time.time())
                
    print (len(X_aug_images) ,len(Y_steer_angles))
    pickle.dump({'X_aug_images':X_aug_images , 'Y_steer_angles':Y_steer_angles},open('data_augmented_1.p', 'wb'))
    

In [ ]:
def get_augmented_image(image,ang_range,shear_range,trans_range):
    
    image = mpimg.imread(image)
    image = transform_image(image,ang_range,shear_range,trans_range)
    
    return image 
    

In [ ]:
## Data Augmentation Starts From Here 

def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    #print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def transform_image(img,ang_range,shear_range,trans_range):
    
    '''
    This function transforms images to generate new images.
    The function takes in following arguments,
    1- Image
    2- ang_range: Range of angles for rotation
    3- shear_range: Range of values to apply affine transform to
    4- trans_range: Range of values to apply translations over. 
    
    A Random uniform distribution is used to generate different parameters for transformation
    
    '''
    # Rotation
    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols,ch = img.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = trans_range*np.random.uniform()-trans_range/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])
    pt1 = 5+shear_range*np.random.uniform()-shear_range/2
    pt2 = 20+shear_range*np.random.uniform()-shear_range/2
    
    # Brightness 
    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])
    shear_M = cv2.getAffineTransform(pts1,pts2)
        
    img = cv2.warpAffine(img,Rot_M,(cols,rows))
    img = cv2.warpAffine(img,Trans_M,(cols,rows))
    img = cv2.warpAffine(img,shear_M,(cols,rows))
    
    img = augment_brightness_camera_images(img)
    
    return img

In [ ]:
if __name__ == "__main__": 
    
    os.chdir('/home/shashiworkspace/Udacity/Term1_Proj3/data/data/')
    pre_process_image()
    
    